In [1]:
%defaultDatasource jdbc:h2:mem:db

In [ ]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes_fcid;
DROP TABLE IF EXISTS Intake;
DROP TABLE IF EXISTS Foods_fcid;
DROP TABLE IF EXISTS Nutrients;
DROP TABLE IF EXISTS Components;
DROP TABLE IF EXISTS Foods_fdb;
DROP TABLE IF EXISTS Contents;
DROP TABLE IF EXISTS EFSA_Intake;


-- Tabelas FCID

CREATE TABLE Crop_Group (
  CGN TINYINT,
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('bases/fcid/FCID_Cropgroup_Description.csv');


-- Tabela com a descrição dos produtos alimentares utilizados nas receitas
CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('data/processed/fcid/FCID_Code_Description.csv');

-- Tabela de cmidas que são descritas na tabela de receitas
CREATE TABLE Foods_fcid (
  Food_Code VARCHAR(8),
  Food_Abbrev_Desc VARCHAR(55),
  Food_Desc VARCHAR(104)
) AS SELECT
  Food_Code, Food_Abbrev_Desc, Food_Desc
FROM CSVREAD('data/processeds/fcid/Food_Code_Description.csv');

-- Tabela de receitas cortada para viabilizar a análise no Binder
CREATE TABLE Recipes_fcid (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(8, 2),
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(Food_Code)
    REFERENCES Foods_fcid(Food_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight
FROM CSVREAD('data/processed/fcid/Recipes_WWEIA_FCID_0510_cropped.csv');

-- Tabela de consumo cortada para viabilizar a análise no binder
CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(15,7),
  Intake_BW DECIMAL(15,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('data/processed/fcid/Commodity_CSFFM_Intake_0510_cropped.csv');


-- Tabelas do FoodDB
CREATE TABLE Nutrients (
  id TINYINT,
  public_id VARCHAR(12),
  name VARCHAR(55),
  description VARCHAR(150),
  PRIMARY KEY(id)
  ) AS SELECT
  id, public_id, name, description
FROM CSVREAD('data/processed/foodb/Nutrients.csv');

CREATE TABLE Compound (
  id TINYINT,
  public_id VARCHAR(12),
  name VARCHAR(55),
  PRIMARY KEY(id)
  ) AS SELECT
  id, public_id, name
FROM CSVREAD('bdata/processed/foodb/Compound.csv');

CREATE TABLE Foods_fdb (
id INT,
public_id VARCHAR(12),
name VARCHAR(55),
name_scientific VARCHAR(55),
description VARCHAR(150),
food_group VARCHAR(55),
food_subgroup VARCHAR(55),
food_type VARCHAR(55),
PRIMARY KEY(id)
  ) AS SELECT
  id, public_id, name, name_scientific, description, food_group, food_subgroup, food_type 
FROM CSVREAD('data/processed/foodb/Food.csv');


CREATE TABLE Contents (
id INT,
source_id INT,
source_type VARCHAR(20),
food_id INT,
orig_food_id INT ,
orig_food_id_commo_name VARCHAR(55),
orig_food_scientific_name VARCHAR(55),
orig_food_part VARCHAR(55),
orig_source_id VARCHAR(55),
orig_source_name VARCHAR(55),
orig_content DOUBLE(10,2),
orig_min DOUBLE(10,2),
orig_max DOUBLE(10,2),
orig_unit varchar(12),
PRIMARY KEY(id),
FOREIGN KEY(food_id)
    REFERENCES Foods_fdb(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
  ) AS SELECT
  id, public_id, name, name_scientific, description, food_group, food_subgroup, food_type 
FROM CSVREAD('data/processed/foodb/Contents.csv');

CREATE TABLE EFSA_Intake(
  Country VARCHAR(12),
  Year TINYINT,
  Population_Group VARCHAR(12),
  Exposure_Hierarchy_1 VARCHAR(55),
  Exposure_Hierarchy_7 VARCHAR(55),
  Number_of_Subjects INT,
  Number_of_Consumers DOUBLE(5,5), 
  Number_of_Consumers/Number_of_Subjects AS  Percent_of_Consumers,
  Intake_Mean DOUBLE(5,5),
  Intake_Mean_SI DOUBLE(5,5),
  Mean_consumers_BW DOUBLE(5,5),
  Mean_Subjects_BW DOUBLE(5,5)  
  ) AS SELECT
  Country, Year, Population_Group, 
  Exposure_Hierarchy_1, Exposure_Hierarchy_7, Percent_of_Consumers, Intake_Mean, Intake_Mean_SI, Mean_consumers_BW, Mean_Subjects_BW
FROM CSVREAD('data/processed/efsa/EFSA_Intake.csv'); 





In [ ]:
SELECT ntrs.calories_per_gram,   cnts.*
FROM Contents as cnts
    join Nutriens as ntrs ON cnts.source_id = ntrs.id
                            AND  cnts.source_id = 'Nutrients'